In [7]:
import csv
import datetime
import h5py
import keras
import numpy as np
import os
import pandas as pd
import sys
import tensorflow as tf
import time

sys.path.append("../src")
import localmodule


# Define constants.
data_dir = localmodule.get_data_dir()
dataset_name = localmodule.get_dataset_name()
folds = localmodule.fold_units()
models_dir = localmodule.get_models_dir()
n_input_hops = 104
bg_duration = 1800


# Read command-line arguments.
args = ["all-but-noise", "unit01", "trial-0", "unit01"]
aug_kind_str = args[0]
test_unit_str = args[1]
trial_str = args[2]
predict_unit_str = args[3]


# Retrieve fold such that unit_str is in the test set.
fold = [f for f in folds if test_unit_str in f[0]][0]
test_units = fold[0]
training_units = fold[1]
validation_units = fold[2]


# Print header.
start_time = int(time.time())
print(str(datetime.datetime.now()) + " Start.")
print("Using NTT-like convnet for binary classification in " +
    dataset_name + " clips with logmelspec input. ")
print("Training set: " + ", ".join(training_units) + ".")
print("Validation set: " + ", ".join(validation_units) + ".")
print("Test set: " + ", ".join(test_units) + ".")
print("")
print('h5py version: {:s}'.format(h5py.__version__))
print('keras version: {:s}'.format(keras.__version__))
print('numpy version: {:s}'.format(np.__version__))
print('tensorflow version: {:s}'.format(tf.__version__))
print("")


# Load model.
model_name = "icassp-ntt-convnet"
if not aug_kind_str == "none":
    model_name = "_".join([model_name, "aug-" + aug_kind_str])
model_dir = os.path.join(models_dir, model_name)
unit_dir = os.path.join(model_dir, test_unit_str)
trial_dir = os.path.join(unit_dir, trial_str)
network_name = "_".join(
    [dataset_name, model_name, test_unit_str, trial_str, "network"])
network_path = os.path.join(trial_dir, network_name + ".hdf5")
model = keras.models.load_model(network_path)


# Open logmelspec container with h5py.
aug_str = "original"
logmelspec_dir = os.path.join(data_dir,
    "_".join([dataset_name, "clip-logmelspec"]))
original_logmelspec_dir = os.path.join(logmelspec_dir, aug_str)
hdf5_name = "_".join([dataset_name, aug_str, predict_unit_str])
hdf5_path = os.path.join(original_logmelspec_dir, hdf5_name + ".hdf5")
lms_container = h5py.File(hdf5_path, "r")
lms_group = lms_container["logmelspec"]
keys = list(lms_group.keys())


# Create HDF5 container for predictions.
clip_predictions_name = "_".join([
    dataset_name,
    model_name,
    "test-" + test_unit_str,
    "predict-" + predict_unit_str,
    "clip-predictions"
])


# Create CSV file.
prediction_name = "_".join([dataset_name, model_name,
    "test-" + test_unit_str, trial_str, "predict-" + predict_unit_str,
    "clip-predictions"])
prediction_path = os.path.join(trial_dir, prediction_name + ".csv")
csv_file = open(prediction_path, 'w')
csv_writer = csv.writer(csv_file, delimiter=',')


# Create CSV header.
csv_header = ["Dataset", "Test unit", "Prediction unit", "Timestamp",
    "Key", "Predicted probability"]
csv_writer.writerow(csv_header)

2017-12-12 19:34:35.711788 Start.
Using NTT-like convnet for binary classification in BirdVox-70k clips with logmelspec input. 
Training set: unit02, unit03, unit05.
Validation set: unit07, unit10.
Test set: unit01.

h5py version: 2.6.0
keras version: 2.0.6
numpy version: 1.13.1
tensorflow version: 1.2.1



71

In [25]:
key = keys[0]

# Load logmelspec.
X_lms = lms_group[key]

# Trim logmelspec in time to required number of hops.
X_width = X_lms.shape[1]
first_col = int((X_width-n_input_hops) / 2)
last_col = int((X_width+n_input_hops) / 2)
X_lms = X_lms[:, first_col:last_col]

# Add trailing singleton dimension for Keras interoperability.
X_lms = X_lms[np.newaxis, :, :, np.newaxis]

# Predict.
#predicted_probability = model.predict(X)[0, 0]

In [24]:
bg_key = "_".join(key.split("_")[:-1])

In [33]:
# Open background container with h5py.
bg_name = "_".join(
    [dataset_name, "clip-logmelspec-backgrounds"])
bg_dir = os.path.join(data_dir, bg_name)
T_str = "T-" + str(bg_duration).zfill(4)
T_dir = os.path.join(bg_dir, T_str)
bg_name = "_".join(
    [dataset_name, "background_summaries",
     test_unit_str, T_str + ".hdf5"])
bg_path = os.path.join(T_dir, bg_name)
bg_container = h5py.File(bg_path, "r")
bg_container

<HDF5 file "BirdVox-70k_background_summaries_unit01_T-1800.hdf5" (mode r)>